In [1]:
import pandas as pd
import requests

In [9]:
r = requests.get("https://gasprices.aaa.com/state-gas-price-averages/", headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"})
results=pd.read_html(r.text)
gas_table=results[0]
gas_table

,State,Regular,Mid-Grade,Premium,Diesel
0,Alaska,$2.381,$2.474,$2.643,$2.366
1,Alabama,$1.763,$2.079,$2.409,$2.272
2,Arkansas,$1.765,$2.027,$2.317,$2.116
3,Arizona,$2.232,$2.509,$2.761,$2.500
4,California,$2.968,$3.143,$3.255,$3.280
5,Colorado,$2.260,$2.553,$2.815,$2.277
6,Connecticut,$2.040,$2.410,$2.641,$2.730
7,District of Columbia,$2.176,$2.693,$2.832,$2.842
8,Delaware,$1.986,$2.383,$2.572,$2.307
9,Florida,$1.995,$2.336,$2.620,$2.430
